In [1]:
########################################################################################
# Abraham Tishelman-Charny                                                             #
# 4 January 2021                                                                       #
#                                                                                      #
# The purpose of this notebook is to plot variables for different systematics trees    #
# to verify variations and scale factors make sense.                                   #
########################################################################################

In [2]:
import ROOT
import os 
import sys 
from array import array
import uproot 
from matplotlib import pyplot as plt 
import numpy as np 

Welcome to JupyROOT 6.20/06


In [3]:
##-- Setup Misc. Parameters
outputLoc = '/eos/user/a/atishelm/www/HHWWgg/Pre-Production-Checks/Systematics/'
verbose = 0

In [4]:
##-- Define Systematic Labels

phosystlabels = []
jetsystlabels = []
metsystlabels = []

##-- Granular JECs
listOfJECsources = [       "Absolute",
                           "Absolute2017",
                           "BBEC1",
                           "BBEC12017",
                           "EC2",
                           "EC22017",
                           "FlavorQCD",
                           "HF",
                           "HF2017",
                           "RelativeBal",
                           "RelativeSample2017"
                    ]
    
phosystlabels.append("MvaShift")
phosystlabels.append("SigmaEOverEShift")
phosystlabels.append("MaterialCentralBarrel")
phosystlabels.append("MaterialOuterBarrel")
phosystlabels.append("MaterialForward")
phosystlabels.append("FNUFEB")
phosystlabels.append("FNUFEE")
phosystlabels.append("MCScaleGain6EB")
phosystlabels.append("MCScaleGain1EB")
jetsystlabels.append("JEC")
jetsystlabels.append("JER")
jetsystlabels.append("PUJIDShift")
for sourceName in listOfJECsources:
    jetsystlabels.append("JEC%s" % (str(sourceName)))
metsystlabels.append("metJecUncertainty")
metsystlabels.append("metJerUncertainty")
metsystlabels.append("metPhoUncertainty")
metsystlabels.append("metUncUncertainty")

for r9 in ["HighR9","LowR9"]:
    for region in ["EB","EE"]:
        phosystlabels.append("ShowerShape%s%s"%(r9,region))
        phosystlabels.append("MCScale%s%s" % (r9,region))
        for var in ["Rho","Phi"]:
            phosystlabels.append("MCSmear%s%s%s" % (r9,region,var))    

if(verbose):
    print("--------------------------")
    print("Photon Systematics Labels:")
    for phosystlabel in phosystlabels:
        print(phosystlabel)
    print("--------------------------")
    print("Jet Systematics Labels:")
    for jetsystlabel in jetsystlabels:
        print(jetsystlabel)
    print("--------------------------")
    print("MET Systematics Labels:")
    for metsystlabel in metsystlabels:
        print(metsystlabel)    
    print("--------------------------")    

print("Number of Photon Systematics Labels:",len(phosystlabels))
print("Number of Jet Systematics Labels:",len(jetsystlabels))
print("Number of MET Systematics Labels:",len(metsystlabels))

allSystLabels = []
for phosyst in phosystlabels:
    allSystLabels.append(phosyst)
for jetsyst in jetsystlabels:
    allSystLabels.append(jetsyst)
for metsyst in metsystlabels:
    allSystLabels.append(metsyst)

print("Number of Systematic Labels:",len(allSystLabels))

Number of Photon Systematics Labels: 25
Number of Jet Systematics Labels: 14
Number of MET Systematics Labels: 4
Number of Systematic Labels: 43


In [5]:
ScaleFactors = [
    "LooseMvaSF", ##-- Doesn't match AN-19-149. 0 uncertainty in 17, 18 
    "PreselSF", ##-- identical for 2017, 2018. 16' different. 
    "electronVetoSF",
    "TriggerWeight",
#     "FracRVWeight", ##-- Not used for HH->WWgg 
    "MuonIDWeight",
    "ElectronIDWeight",
    "ElectronRecoWeight",
    "MuonIsoWeight",
    "JetBTagCutWeight",
    "JetBTagReshapeWeight",
    "prefireWeight"
]

TheoryUncertainties = [
    "THU_ggH_Mu",
    "THU_ggH_Res",
    "THU_ggH_Mig01",
    "THU_ggH_Mig12",
    "THU_ggH_VBF2j",
    "THU_ggH_VBF3j",
    "THU_ggH_PT60",
    "THU_ggH_PT120",
    "THU_ggH_qmtop"
]

In [163]:
array1 = np.array([1,2,3])
array2 = np.array([2,1,4])

array3 = array1 / array2

# array3 = array1 > array2 

# print(array3)

# print(len(array3))

# newArray = array3[array3 == 1]
# # print(len(newArray))

# print(array3[0])

In [166]:
PhotonSFs = [
    "LooseMvaSF", ##-- Doesn't match AN-19-149. 0 uncertainty in 17, 18 
    "PreselSF", ##-- identical for 2017, 2018. 16' different. 
    "electronVetoSF",
    "TriggerWeight",           
]

print("LooseMvaSF" in PhotonSFs)
print("testest" in PhotonSFs)

True
False


In [168]:
# https://uproot.readthedocs.io/en/latest/basic.html

Direc = "/eos/user/a/atishelm/ntuples/HHWWgg_flashgg/Pre-Production-Checks/PDFWeights/hadded/"
# Direc = "/eos/user/a/atishelm/ntuples/HHWWgg_flashgg/Pre-Production-Checks/PDFWeights/FL-50kEvents/"
files = [os.path.join(Direc, file) for file in os.listdir(Direc)]
for file in files:
    print("file:",file) 
    file_uproot = uproot.open(file)
    channelDict = {
        "SL" : ["SL","tagsDumper/trees/GluGluToHHTo2G2Qlnu_node_cHHH1_13TeV_HHWWggTag_0"],
        "FH" : ["FH","tagsDumper/trees/GluGluToHHTo2G4Q_node_cHHH1_13TeV_HHWWggTag_1"],
        "FL" : ["FL","tagsDumper/trees/GluGluToHHTo2G2l2nu_node_cHHH1_13TeV_HHWWggTag_2"]
    }
    
    for key in channelDict:
        if(key in file):
            channel, NominalTreename = channelDict[key]
    print("channel:",channel)
    print("NominalTreename:",NominalTreename)
    xmin, xmax, xbins = 115, 135, 20
    bins = np.linspace(xmin, xmax, xbins + 1) ##-- xbins + 1 for proper number of boundaries in linear space     

    ##-- Scaling
    HH_XS = 31.049 ##-- pb 
    HH_WWgg_BR = 0.00097
    LUMI = 41.5 
    
    WWgg_BRs = {
        "SL" : 0.441, 
        "FH" : 0.454,
        "FL" : 0.107
    }
    
    WWgg_BR = WWgg_BRs[channel]
    

    SCALE = float(LUMI) * float(HH_XS) * float(HH_WWgg_BR) * float(WWgg_BR)
    print("SCALE:",SCALE)

    ##-- Get Nominal tree and weights
    CMS_hgg_mass_nominal = file_uproot[NominalTreename]["CMS_hgg_mass"].array()
    weights_nominal = file_uproot[NominalTreename]["weight"].array()
    weights_nominal = weights_nominal * SCALE

    PhotonSFs = [
        "LooseMvaSF", ##-- Doesn't match AN-19-149. 0 uncertainty in 17, 18 
        "PreselSF", ##-- identical for 2017, 2018. 16' different. 
        "electronVetoSF",
        "TriggerWeight",           
    ]
 
    PhotonCentralWeights = file_uproot[NominalTreename]["DiphoCentralWeight"].array()
    prefireWeightCentral = file_uproot[NominalTreename]["prefireWeightCentral"].array()
    ##-- Plot Scale Factors in nominal tree
    xmin, xmax, xbins = 0.25, 1.75, 40
    bins = np.linspace(xmin, xmax, xbins + 1) ##-- xbins + 1 for proper number of boundaries in linear space   
    for ScaleFactor in ScaleFactors:
        print("On SF:",ScaleFactor)
        SF_Central = "%sCentral"%(ScaleFactor)
        SF_Up = "%sUp01sigma"%(ScaleFactor)
        SF_Down = "%sDown01sigma"%(ScaleFactor)
        
        SF_Central_vals = file_uproot[NominalTreename][SF_Central].array()
        SF_Up_vals = file_uproot[NominalTreename][SF_Up].array()
        SF_Down_vals = file_uproot[NominalTreename][SF_Down].array()
        
#         if(ScaleFactor != "prefireWeight"):
        if(ScaleFactor in PhotonSFs):
            SF_Up_vals = SF_Up_vals / prefireWeightCentral
            SF_Down_vals = SF_Down_vals / prefireWeightCentral
            SF_Up_vals / PhotonCentralWeights
            SF_Down_vals / PhotonCentralWeights
        
        ##-- Check order of down, central, up values
#         OrderMakesSenseTotal = ((SF_Down_vals < SF_Central_vals) and (SF_Central_vals < SF_Up_vals))
        Down_comp = (SF_Down_vals < SF_Central_vals)
        Up_comp = (SF_Central_vals < SF_Up_vals)
        
        
#         OrderMakesSenseTotal = (Down_comp==1 and Up_comp==1)
#         OrderMakesSenseTotal = np.array([])
        OrderMakesSenseTotal = []
    
        for ientry,Central in enumerate(SF_Central_vals):
            down =  SF_Down_vals[ientry]
            up = SF_Up_vals[ientry]
            if((float(down) < float(Central)) and (float(Central) < float(up)) ):
                OrderMakesSenseTotal.append(1)
            else:
                OrderMakesSenseTotal.append(0)
        OrderMakesSenseTotalArr = np.array(OrderMakesSenseTotal)
            
        denom = len(OrderMakesSenseTotalArr)
        
        OrderMakesSense = OrderMakesSenseTotalArr[OrderMakesSenseTotalArr == 1]
        
        numerator = len(OrderMakesSense)
        
        print("num:",numerator)
        print("denom:",denom)
        print("Percent that makes sense:",float(numerator) / float(denom))
    
        fig, ax = plt.subplots()
        plt.hist(SF_Central_vals,
                 bins = bins,
                 histtype = 'step',
                 label = "CENTRAL"
                )
        plt.hist(SF_Up_vals,
                 bins = bins,
                 histtype = 'step',
                 label = "UP"
                )
        plt.hist(SF_Down_vals,
                 bins = bins,
                 histtype = 'step',
                 label = "DOWN"
                )  
        
        ax.set_title("%s: %s"%(channel, ScaleFactor))
        ax.set_xlabel(ScaleFactor)
        ax.set_ylabel("Entries")
        plt.legend()
        plt.savefig("%s/%s/ScaleFactors/%s.png"%(outputLoc,channel,ScaleFactor))
        plt.close()
        
        
        
    
#     ##-- Plot CMS_hgg_mass for nominal, +/- 1 Sigma Systematics 
#     for iLabel,SystLabel in enumerate(allSystLabels):
#         print("On Systematic %s / %s: %s"%(iLabel,len(allSystLabels)-1,SystLabel))
#         up_tree = "%s_%sUp01sigma"%(NominalTreename,SystLabel)
#         down_tree = "%s_%sDown01sigma"%(NominalTreename,SystLabel)

#         mass_up = file_uproot[up_tree]["CMS_hgg_mass"].array()
#         mass_down = file_uproot[down_tree]["CMS_hgg_mass"].array()

#         weights_up = file_uproot[up_tree]["weight"].array()
#         weights_down = file_uproot[down_tree]["weight"].array()

#         weights_up = weights_up * SCALE
#         weights_down = weights_down * SCALE

#         fig, ax = plt.subplots()
#         plt.hist(CMS_hgg_mass_nominal,
#                  weights = weights_nominal,
#                  bins = bins,
#                  histtype = 'step',
#                  label = 'NOMINAL'
#                 )
#         plt.hist(mass_up ,
#                  weights = weights_up,
#                  bins = bins,
#                  histtype = 'step',
#                  label = 'Up 1 Sigma'
#                  )
#         plt.hist(mass_down,
#                  weights = weights_down,
#                  bins = bins,
#                  histtype = 'step',
#                  label = "Down 1 Sigma"
#                  )             
#         ax.set_title("CMS_hgg_mass, SystLabel: %s"%(SystLabel))
#         ax.set_xlabel("CMS_hgg_mass")
#         ax.set_ylabel("Entries")
#         plt.legend()
#         plt.savefig("%s/%s/CMS_hgg_mass-%s.png"%(outputLoc,channel,SystLabel))
#         plt.close()
    

file: /eos/user/a/atishelm/ntuples/HHWWgg_flashgg/Pre-Production-Checks/PDFWeights/hadded/FH-50kEvents.root
channel: FH
NominalTreename: tagsDumper/trees/GluGluToHHTo2G4Q_node_cHHH1_13TeV_HHWWggTag_1
SCALE: 0.56744438273
On SF: LooseMvaSF
num: 0
denom: 10759
Percent that makes sense: 0.0
On SF: PreselSF
num: 1463
denom: 10759
Percent that makes sense: 0.13597918022121014
On SF: electronVetoSF
num: 0
denom: 10759
Percent that makes sense: 0.0
On SF: TriggerWeight
num: 0
denom: 10759
Percent that makes sense: 0.0
On SF: MuonIDWeight
num: 0
denom: 10759
Percent that makes sense: 0.0
On SF: ElectronIDWeight
num: 0
denom: 10759
Percent that makes sense: 0.0
On SF: ElectronRecoWeight
num: 0
denom: 10759
Percent that makes sense: 0.0
On SF: MuonIsoWeight
num: 0
denom: 10759
Percent that makes sense: 0.0
On SF: JetBTagCutWeight
num: 0
denom: 10759
Percent that makes sense: 0.0
On SF: JetBTagReshapeWeight


/usr/local/bin/start_ipykernel.py:76: RuntimeWarning: invalid value encountered in less
/usr/local/bin/start_ipykernel.py:77: RuntimeWarning: invalid value encountered in less


num: 3193
denom: 10759
Percent that makes sense: 0.2967747931963937
On SF: prefireWeight
num: 5287
denom: 10759
Percent that makes sense: 0.4914025467050841
file: /eos/user/a/atishelm/ntuples/HHWWgg_flashgg/Pre-Production-Checks/PDFWeights/hadded/FL-50kEvents_filesmissing.root
channel: FL
NominalTreename: tagsDumper/trees/GluGluToHHTo2G2l2nu_node_cHHH1_13TeV_HHWWggTag_2
SCALE: 0.133736891965
On SF: LooseMvaSF
num: 0
denom: 1912
Percent that makes sense: 0.0
On SF: PreselSF
num: 243
denom: 1912
Percent that makes sense: 0.12709205020920503
On SF: electronVetoSF
num: 0
denom: 1912
Percent that makes sense: 0.0
On SF: TriggerWeight
num: 0
denom: 1912
Percent that makes sense: 0.0
On SF: MuonIDWeight
num: 22
denom: 1912
Percent that makes sense: 0.011506276150627616
On SF: ElectronIDWeight
num: 498
denom: 1912
Percent that makes sense: 0.2604602510460251
On SF: ElectronRecoWeight
num: 356
denom: 1912
Percent that makes sense: 0.18619246861924685
On SF: MuonIsoWeight
num: 0
denom: 1912
Perc

In [115]:
# xmin, xmax, xbins = 115, 135, 20
# bins = np.linspace(xmin, xmax, xbins + 1) ##-- xbins + 1 for proper number of boundaries in linear space 

# HH_XS = 31.049 ##-- pb 
# HH_WWgg_BR = 0.00097
# LUMI = 41.5 
# WWgg_SL_BR = 0.441
# WWgg_FH_BR = 0.454
# WWgg_FL_BR = 0.107

# SCALE = float(LUMI) * float(HH_XS) * float(HH_WWgg_BR) * float(WWgg_SL_BR)

# # SCALE = "%d*%d*%d"%(LUMI,HH_XS,HH_WWgg_BR,WWgg_SL_BR) ##-- SL BR included
# print("SCALE:",SCALE)

# CMS_hgg_mass_nominal = SL_File["tagsDumper/trees"][local_SL_treeName_nominal]["CMS_hgg_mass"].array()
# weights_nominal = SL_File["tagsDumper/trees"][local_SL_treeName_nominal]["weight"].array()
# weights_nominal = weights_nominal * SCALE

# ##-- Plot CMS_hgg_mass for nominal, +/- 1 Sigma Systematics 
# for iLabel,SystLabel in enumerate(allSystLabels):
#     print("On Systematic %s / %s: %s"%(iLabel,len(allSystLabels)-1,SystLabel))
#     up_tree = "%s_%sUp01sigma"%(local_SL_treeName_nominal,SystLabel)
#     down_tree = "%s_%sDown01sigma"%(local_SL_treeName_nominal,SystLabel)
    
#     mass_up = SL_File["tagsDumper/trees"][up_tree]["CMS_hgg_mass"].array()
#     mass_down = SL_File["tagsDumper/trees"][down_tree]["CMS_hgg_mass"].array()
    
#     weights_up = SL_File["tagsDumper/trees"][up_tree]["weight"].array()
#     weights_down = SL_File["tagsDumper/trees"][down_tree]["weight"].array()
    
#     weights_up = weights_up * SCALE
#     weights_down = weights_down * SCALE
    
#     fig, ax = plt.subplots()
#     plt.hist(CMS_hgg_mass_nominal,
#              weights = weights_nominal,
#              bins = bins,
#              histtype = 'step',
#              label = 'NOMINAL'
#             )
#     plt.hist(mass_up ,
#              weights = weights_up,
#              bins = bins,
#              histtype = 'step',
#              label = 'Up 1 Sigma'
#              )
#     plt.hist(mass_down,
#              weights = weights_down,
#              bins = bins,
#              histtype = 'step',
#              label = "Down 1 Sigma"
#              )             
#     ax.set_title("CMS_hgg_mass, SystLabel: %s"%(SystLabel))
#     ax.set_xlabel("CMS_hgg_mass")
#     ax.set_ylabel("Entries")
#     plt.legend()
#     plt.savefig("%s/CMS_hgg_mass-%s.png"%(outputLoc,SystLabel))
#     plt.close()
#     if(iLabel==2): break  
        
##-- Plot PDF, AlphaS, Scale weights for nominal branch only 
# PDFxmin, PDFxmax, PDFxbins = 0,2,20
PDFxmin, PDFxmax, PDFxbins = 0,10,200
PDFbins = np.linspace(PDFxmin, PDFxmax, PDFxbins + 1) ##-- xbins + 1 for proper number of boundaries in linear space 

nPdfWeights = 60
nAlphaSWeights = 2
nScaleWeights = 9

allPDFWeights = SL_File["tagsDumper/trees"][local_SL_treeName_nominal]["pdfWeights"].array()
allAlphaSWeights = SL_File["tagsDumper/trees"][local_SL_treeName_nominal]["alphaSWeights"].array() 
allscaleWeights = SL_File["tagsDumper/trees"][local_SL_treeName_nominal]["scaleWeights"].array() 

pdfweight_avgs = []

# for PdfWeight_i in range(nPdfWeights):
#     print("On Pdf Weight %s / %s "%(PdfWeight_i,nPdfWeights-1))
#     PDFWeights = allPDFWeights[:,PdfWeight_i]
#     print("avg:",PDFWeights.mean())
#     pdfweight_avgs.append(PDFWeights.mean())
    
#     fig, ax = plt.subplots()
#     plt.hist(PDFWeights,
#              bins = PDFbins,
#              histtype = 'step',
#              label = 'NOMINAL'
#             )     
#     ax.set_title("PDF Weight %s"%(PdfWeight_i))
#     ax.set_xlabel("PDF Weight %s"%(PdfWeight_i))
#     ax.set_ylabel("Entries")
#     plt.legend()
#     plt.savefig("%s/PDFWeights/PDFWeights_%s.png"%(outputLoc,PdfWeight_i))
#     plt.close()
    
# ##-- Averages
# fig, ax = plt.subplots()    
# plt.hist(pdfweight_avgs,
#          histtype = 'step',
#          label = 'PDF Weight Averages'
#         )
# plt.legend()
# plt.savefig("%s/PDFWeights/PDFWeights_Averages.png"%(outputLoc))

# fig, ax = plt.subplots()
    
# for AlphaSWeight_i in range(nAlphaSWeights):
#     print("On AlphaS Weight %s / %s "%(AlphaSWeight_i,nAlphaSWeights-1))
#     AlphaSWeights = allAlphaSWeights[:,AlphaSWeight_i]
#     print("len(AlphaSWeights)",len(AlphaSWeights))
    
#     plt.hist(AlphaSWeights,
#              bins = PDFbins,
#              histtype = 'step',
#              label = 'Alpha S weight %s'%(AlphaSWeight_i)
#             )     
# ax.set_title("Alpha S Weights")
# ax.set_xlabel("Alpha S Weights")
# ax.set_ylabel("Entries")
# plt.legend()
# plt.savefig("%s/AlphaSWeights/AlphaSWeights.png"%(outputLoc))
# plt.close() 
    
fig, ax = plt.subplots()

for ScaleWeight_i in range(nScaleWeights):
    print("On Scale Weight %s / %s "%(ScaleWeight_i,nScaleWeights-1))
    ScaleWeights = allscaleWeights[:,ScaleWeight_i]
    plt.hist(ScaleWeights,
             bins = PDFbins,
             histtype = 'step',
             label = 'Scale Weight %s'%(ScaleWeight_i)
            )     
    
ax.set_title("Scale Weights")
ax.set_xlabel("Scale Weights")
ax.set_ylabel("Entries")
plt.legend()
plt.savefig("%s/ScaleWeights/ScaleWeights.png"%(outputLoc))
plt.close()     

##-- Plot Theory Weights


##-- Plot Scale Factors. Maybe for both nominal and variations  


On Scale Weight 0 / 8 
On Scale Weight 1 / 8 
On Scale Weight 2 / 8 
On Scale Weight 3 / 8 
On Scale Weight 4 / 8 
On Scale Weight 5 / 8 
On Scale Weight 6 / 8 
On Scale Weight 7 / 8 
On Scale Weight 8 / 8 


In [ ]:
-------------- DUMPING SYSTEMATIC OVERVIEW --------------
          Systematic  Central value?   Systematic shifts?
---------------------------------------------------------
      MCScaleLowR9EE              NO                -1 1 
     MCScaleHighR9EE              NO                -1 1 
      MCScaleLowR9EB              NO                -1 1 
     MCScaleHighR9EB              NO                -1 1 
      MCScaleGain6EB              NO                -1 1 
      MCScaleGain1EB              NO                -1 1 
MaterialCentralBarrel              NO                -1 1 
 MaterialOuterBarrel              NO                -1 1 
     MaterialForward              NO                -1 1 
 ShowerShapeHighR9EB              NO                -1 1 
 ShowerShapeHighR9EE              NO                -1 1 
  ShowerShapeLowR9EB              NO                -1 1 
  ShowerShapeLowR9EE              NO                -1 1 
              FNUFEB              NO                -1 1 
              FNUFEE              NO                -1 1 
            MvaShift              NO                -1 1 
            PreselSF             YES                -1 1 
      electronVetoSF             YES                -1 1 
       TriggerWeight             YES                -1 1 
          LooseMvaSF             YES                -1 1 
 SigmaEOverESmearing             YES                   NO
    SigmaEOverEShift              NO                -1 1 
---------------------------------------------------------
   MuonTightIDWeight             YES                -1 1 
MuonTightRelISOWeight             YES                -1 1 
---------------------------------------------------------
    ElectronIDWeight             YES                -1 1 
  ElectronRecoWeight             YES                -1 1 
---------------------------------------------------------
                 JEC             YES                -1 1 
                 JER             YES                -1 1 
    JetBTagCutWeight             YES                -1 1 
JetBTagReshapeWeight             YES                -1 1 
          PUJIDShift              NO                -1 1 
   UnmatchedPUWeight              NO                -1 1 
         JECAbsolute              NO                -1 1 
     JECAbsolute2017              NO                -1 1 
            JECBBEC1              NO                -1 1 
        JECBBEC12017              NO                -1 1 
              JECEC2              NO                -1 1 
          JECEC22017              NO                -1 1 
        JECFlavorQCD              NO                -1 1 
               JECHF              NO                -1 1 
           JECHF2017              NO                -1 1 
      JECRelativeBal              NO                -1 1 
JECRelativeSample2017              NO                -1 1 
---------------------------------------------------------
------------- DUMPING 2D SYSTEMATIC OVERVIEW ------------
          Systematic  Central value?   Systematic shifts?
---------------------------------------------------------
     MCSmearHighR9EE             YES       1st: 1 -1 0 0 
                                           2nd: 0 0 1 -1 
      MCSmearLowR9EE             YES       1st: 1 -1 0 0 
                                           2nd: 0 0 1 -1 
     MCSmearHighR9EB             YES       1st: 1 -1 0 0 
                                           2nd: 0 0 1 -1 
      MCSmearLowR9EB             YES       1st: 1 -1 0 0 
                                           2nd: 0 0 1 -1 
---------------------------------------------------------